In [1]:
import requests
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
import json
from bs4 import BeautifulSoup
from pygeocoder import Geocoder

In [2]:
def mcd_scrape(url):
    response = requests.get(url)
    S = BeautifulSoup(response.text, 'lxml')
    mcd = []
    list_items = S.find_all('li')
    for L in list_items:
        if "McDonald's in" in L.text:
            addr = L.text.replace("  ","")
            addr = addr.replace("\n","")
            addr = addr.split("McDonald's in")
            mcd.append(addr)
    mcd2 = pd.DataFrame(mcd).drop(0,axis=1).values.tolist()

    mcd3 = []
    for elem in mcd2:
        mcd3.extend(elem)
    mcd4 = []
    for elem in mcd3:
        if elem != None:
            mcd4.append(elem)
    mcd5 = pd.DataFrame(mcd4)
    mcd6 = mcd5[0].str.split(' ',2, expand = True)
    mcd7 = mcd6[2].str.split(' ',1, expand = True).drop(0,axis=1)
    mcd8=mcd6[1].values.tolist()
    mcd9=mcd7.values.tolist()
    mcd9_list = []
    for elem in mcd9:
        mcd9_list.extend(elem)
    mcd10 = pd.DataFrame()
    mcd10['address'] = mcd9_list
    mcd10['zip'] = mcd8
    return(mcd10)

In [3]:
mo_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/mcdonalds-21019/us/mo")
mo_mcd['state'] = "MO"
mo_mcd

,address,zip,state
0,3996 Jeffco Blvd,63010,MO
1,130 Arnold Crossroads Ctr,63010,MO
2,2201 Michigan Ave,63010,MO
3,2005 Richardson Rd,63010,MO
4,3996 Jeffco Blvd,63010,MO
...,...,...,...
583,1315 Preacher Roe Blvd,65775,MO
584,602 S Harris St,65793,MO
585,607 N Harris St,65793,MO
586,602 S Harris St,65793,MO


In [4]:
il_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/mcdonalds-21019/us/il")
il_mcd['state'] = "IL"
il_mcd

,address,zip,state
0,501 W Lake St,60101-2306,IL
1,410 W North Ave,60101-4912,IL
2,501 W Lake St,60101-2306,IL
3,410 W North Ave,60101-4912,IL
4,125 S Randall Rd,60102,IL
...,...,...,...
1225,12750 New York 34,60560,IL
1226,2028 Sheridan Rd,60099,IL
1227,2008 N Sheridan Rd,60099-4119,IL
1228,2028 Sheridan Rd,60099,IL


In [5]:
both = [mo_mcd,il_mcd]
both_mcd = pd.concat(both)
cleaned_mcd = pd.DataFrame(both_mcd['zip'].str.split('-').str[0])
cleaned_mcd['address']= both_mcd['address']
cleaned_mcd['state'] = both_mcd['state']
cleaned_mcd = cleaned_mcd[['address', 'state', 'zip']]
cleaned_mcd

,address,state,zip
0,3996 Jeffco Blvd,MO,63010
1,130 Arnold Crossroads Ctr,MO,63010
2,2201 Michigan Ave,MO,63010
3,2005 Richardson Rd,MO,63010
4,3996 Jeffco Blvd,MO,63010
...,...,...,...
1225,12750 New York 34,IL,60560
1226,2028 Sheridan Rd,IL,60099
1227,2008 N Sheridan Rd,IL,60099
1228,2028 Sheridan Rd,IL,60099


In [6]:
cleaned_mcd['location'] = cleaned_mcd[['address', 'state', 'zip']].agg(' '.join, axis=1)
cleaned_mcd = cleaned_mcd.drop_duplicates()
cleaned_mcd

,address,state,zip,location
0,3996 Jeffco Blvd,MO,63010,3996 Jeffco Blvd MO 63010
1,130 Arnold Crossroads Ctr,MO,63010,130 Arnold Crossroads Ctr MO 63010
2,2201 Michigan Ave,MO,63010,2201 Michigan Ave MO 63010
3,2005 Richardson Rd,MO,63010,2005 Richardson Rd MO 63010
8,15204 Manchester Rd,MO,63011,15204 Manchester Rd MO 63011
...,...,...,...,...
1220,3 W Veterans Pkwy,IL,60560,3 W Veterans Pkwy IL 60560
1221,2007 Bridge St,IL,60560,2007 Bridge St IL 60560
1222,12750 New York 34,IL,60560,12750 New York 34 IL 60560
1226,2028 Sheridan Rd,IL,60099,2028 Sheridan Rd IL 60099


In [8]:
cleaned_mcd['location'].to_csv('../mcd_loc.csv', index = False)
check = pd.read_csv('../mcd_loc.csv')
check

,location
0,3996 Jeffco Blvd MO 63010
1,130 Arnold Crossroads Ctr MO 63010
2,2201 Michigan Ave MO 63010
3,2005 Richardson Rd MO 63010
4,15204 Manchester Rd MO 63011
...,...
861,3 W Veterans Pkwy IL 60560
862,2007 Bridge St IL 60560
863,12750 New York 34 IL 60560
864,2028 Sheridan Rd IL 60099


In [9]:
# used https://dash.geocod.io/ to geocode data and create new csv with lat/long info
geodata = pd.read_csv('../geocoded_mcd_loc.csv')
geodata

,location,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
0,3996 Jeffco Blvd MO 63010,38.405690,-90.376802,1.00,rooftop,3996.0,Jeffco Blvd,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
1,130 Arnold Crossroads Ctr MO 63010,38.445494,-90.374454,0.88,nearest_rooftop_match,120.0,Arnold Crossroads Ctr,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
2,2201 Michigan Ave MO 63010,38.437040,-90.377478,1.00,rooftop,2201.0,Michigan Ave,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
3,2005 Richardson Rd MO 63010,38.418379,-90.390784,1.00,rooftop,2005.0,Richardson Rd,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
4,3996 Jeffco Blvd MO 63010,38.405690,-90.376802,1.00,rooftop,3996.0,Jeffco Blvd,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,12750 New York 34 IL 60560,41.662109,-88.502787,0.80,rooftop,12750.0,State Rte 34,NaN,NaN,Plano,IL,Kendall County,60545,US,Kendall
1814,2028 Sheridan Rd IL 60099,42.457391,-87.825641,0.89,nearest_rooftop_match,2024.0,Sheridan Rd,NaN,NaN,Zion,IL,Lake County,60099,US,Lake
1815,2008 N Sheridan Rd IL 60099,42.458610,-87.825016,0.90,range_interpolation,2008.0,N Sheridan Rd,NaN,NaN,Zion,IL,Lake County,60099,US,TIGER/Line® dataset from the US Census Bureau
1816,2028 Sheridan Rd IL 60099,42.457391,-87.825641,0.89,nearest_rooftop_match,2024.0,Sheridan Rd,NaN,NaN,Zion,IL,Lake County,60099,US,Lake


In [10]:
#insert code here that drops all rows in cleaned_mcd that don't contain the relevant zip codes

zips = []
with open('../zip_list.txt', 'r') as f:
    zips.extend(f.read().split('\n'))
zips

['63001',
 '63005',
 '63006',
 '63011',
 '63017',
 '63021',
 '63022',
 '63024',
 '63025',
 '63026',
 '63031',
 '63032',
 '63033',
 '63034',
 '63038',
 '63040',
 '63042',
 '63043',
 '63044',
 '63045',
 '63074',
 '63088',
 '63099',
 '63105',
 '63114',
 '63117',
 '63119',
 '63121',
 '63122',
 '63123',
 '63124',
 '63125',
 '63126',
 '63127',
 '63128',
 '63129',
 '63130',
 '63131',
 '63132',
 '63133',
 '63134',
 '63135',
 '63136',
 '63137',
 '63138',
 '63140',
 '63141',
 '63143',
 '63144',
 '63145',
 '63146',
 '63151',
 '63167',
 '63198']

In [11]:
complete_mcd = geodata[~geodata.Zip.isin(zips)]
complete_mcd

,location,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
0,3996 Jeffco Blvd MO 63010,38.405690,-90.376802,1.00,rooftop,3996.0,Jeffco Blvd,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
1,130 Arnold Crossroads Ctr MO 63010,38.445494,-90.374454,0.88,nearest_rooftop_match,120.0,Arnold Crossroads Ctr,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
2,2201 Michigan Ave MO 63010,38.437040,-90.377478,1.00,rooftop,2201.0,Michigan Ave,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
3,2005 Richardson Rd MO 63010,38.418379,-90.390784,1.00,rooftop,2005.0,Richardson Rd,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
4,3996 Jeffco Blvd MO 63010,38.405690,-90.376802,1.00,rooftop,3996.0,Jeffco Blvd,NaN,NaN,Arnold,MO,Jefferson County,63010,US,Jefferson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,12750 New York 34 IL 60560,41.662109,-88.502787,0.80,rooftop,12750.0,State Rte 34,NaN,NaN,Plano,IL,Kendall County,60545,US,Kendall
1814,2028 Sheridan Rd IL 60099,42.457391,-87.825641,0.89,nearest_rooftop_match,2024.0,Sheridan Rd,NaN,NaN,Zion,IL,Lake County,60099,US,Lake
1815,2008 N Sheridan Rd IL 60099,42.458610,-87.825016,0.90,range_interpolation,2008.0,N Sheridan Rd,NaN,NaN,Zion,IL,Lake County,60099,US,TIGER/Line® dataset from the US Census Bureau
1816,2028 Sheridan Rd IL 60099,42.457391,-87.825641,0.89,nearest_rooftop_match,2024.0,Sheridan Rd,NaN,NaN,Zion,IL,Lake County,60099,US,Lake


In [12]:
complete_mcd = complete_mcd[["location", "Accuracy Score", "Latitude", "Longitude", "City", "State", "Zip"]]
complete_mcd = complete_mcd.rename(columns={"location":"id", "Accuracy Score":"strLocation", "Latitude":"latitude", "Longitude":"longitude", "City":"city", "State":"state", "Zip":"zip"})
complete_mcd["Store"] = "McDonald's"
complete_mcd

,id,strLocation,latitude,longitude,city,state,zip,Store
0,3996 Jeffco Blvd MO 63010,1.00,38.405690,-90.376802,Arnold,MO,63010,McDonald's
1,130 Arnold Crossroads Ctr MO 63010,0.88,38.445494,-90.374454,Arnold,MO,63010,McDonald's
2,2201 Michigan Ave MO 63010,1.00,38.437040,-90.377478,Arnold,MO,63010,McDonald's
3,2005 Richardson Rd MO 63010,1.00,38.418379,-90.390784,Arnold,MO,63010,McDonald's
4,3996 Jeffco Blvd MO 63010,1.00,38.405690,-90.376802,Arnold,MO,63010,McDonald's
...,...,...,...,...,...,...,...,...
1813,12750 New York 34 IL 60560,0.80,41.662109,-88.502787,Plano,IL,60545,McDonald's
1814,2028 Sheridan Rd IL 60099,0.89,42.457391,-87.825641,Zion,IL,60099,McDonald's
1815,2008 N Sheridan Rd IL 60099,0.90,42.458610,-87.825016,Zion,IL,60099,McDonald's
1816,2028 Sheridan Rd IL 60099,0.89,42.457391,-87.825641,Zion,IL,60099,McDonald's


In [13]:
complete_mcd.to_csv('../final_mcd_loc.csv', index = False)
check = pd.read_csv('../final_mcd_loc.csv')
check

,id,strLocation,latitude,longitude,city,state,zip,Store
0,3996 Jeffco Blvd MO 63010,1.00,38.405690,-90.376802,Arnold,MO,63010,McDonald's
1,130 Arnold Crossroads Ctr MO 63010,0.88,38.445494,-90.374454,Arnold,MO,63010,McDonald's
2,2201 Michigan Ave MO 63010,1.00,38.437040,-90.377478,Arnold,MO,63010,McDonald's
3,2005 Richardson Rd MO 63010,1.00,38.418379,-90.390784,Arnold,MO,63010,McDonald's
4,3996 Jeffco Blvd MO 63010,1.00,38.405690,-90.376802,Arnold,MO,63010,McDonald's
...,...,...,...,...,...,...,...,...
1663,12750 New York 34 IL 60560,0.80,41.662109,-88.502787,Plano,IL,60545,McDonald's
1664,2028 Sheridan Rd IL 60099,0.89,42.457391,-87.825641,Zion,IL,60099,McDonald's
1665,2008 N Sheridan Rd IL 60099,0.90,42.458610,-87.825016,Zion,IL,60099,McDonald's
1666,2028 Sheridan Rd IL 60099,0.89,42.457391,-87.825641,Zion,IL,60099,McDonald's
